In [15]:
import math
import heapq

# Define directions with index mappings for ease of rotation
DIRECTIONS = ['up', 'right', 'down', 'left']

# Define the Cell class
class Cell:
    def __init__(self):
        self.parent_i = 0  # Parent cell's row index
        self.parent_j = 0  # Parent cell's column index
        self.f = float('inf')  # Total cost of the cell (g + h)
        self.g = float('inf')  # Cost from start to this cell
        self.h = 0  # Heuristic cost from this cell to destination

# Define the size of the grid
rows = int(input("Enter number of rows: "))
cols = int(input("Enter number of columns: "))

# Check if a cell is valid (within the grid)
def is_valid(row, col):
    return (row >= 0) and (row < rows) and (col >= 0) and (col < cols)

# Check if a cell is unblocked
def is_unblocked(grid, row, col):
    return grid[row][col] == '.'

# Check if a cell is the destination
def is_destination(row, col, dest):
    return row == dest[0] and col == dest[1]

# Calculate the heuristic value of a cell (Euclidean distance to destination)
def calculate_h_value(row, col, dest):
    return ((row - dest[0]) ** 2 + (col - dest[1]) ** 2) ** 0.5

# Trace the path from source to destination with direction printing
def trace_path(cell_details, dest):
    print("The Path is ")
    path = []
    row = dest[0]
    col = dest[1]

    # Trace the path from destination to source using parent cells
    while not (cell_details[row][col].parent_i == row and cell_details[row][col].parent_j == col):
        path.append((row, col))
        temp_row = cell_details[row][col].parent_i
        temp_col = cell_details[row][col].parent_j
        row = temp_row
        col = temp_col

    # Add the source cell to the path
    path.append((row, col))
    # Reverse the path to get the path from source to destination
    path.reverse()

    # Print the path
    for i in path:
        print("->", i, end=" ")
    print()

    # Now print the direction of movement
    print("\nDirection of Movement: ")
    orientation = 'up'  # Initial orientation is upward
    for k in range(len(path) - 1):
        curr = path[k]
        next_ = path[k + 1]
        orientation, movement = move_bot(curr, next_, orientation)
        if(movement == "Right() Forward()"):
            print(f"From {curr} to {curr}: {movement.split(" ")[0]}")
            print(f"From {curr} to {next_}: {movement.split(" ")[1]}")
        elif(movement == "Left() Forward()"):
            print(f"From {curr} to {curr}: {movement.split(" ")[0]}")
            print(f"From {curr} to {next_}: {movement.split(" ")[1]}")
        else:   
            print(f"From {curr} to {next_}: {movement}")

# Determine movement and orientation changes
def move_bot(curr, next_, orientation):
    movement = ""
    row_diff = next_[0] - curr[0]
    col_diff = next_[1] - curr[1]

    if row_diff == 0 and col_diff == 1:  # Right
        movement, orientation = move_right(orientation)
    elif row_diff == 0 and col_diff == -1:  # Left
        movement, orientation = move_left(orientation)
    elif row_diff == 1 and col_diff == 0:  # Down
        movement, orientation = move_down(orientation)
    elif row_diff == -1 and col_diff == 0:  # Up
        movement, orientation = move_up(orientation)
    
    return orientation, movement

# Define movement commands based on current orientation
def move_right(orientation):
    if orientation == 'right':
        return 'Forward()', 'right'
    elif orientation == 'up':
        return 'Right() Forward()', 'right'
    elif orientation == 'down':
        return 'Left() Forward()', 'right'
    elif orientation == 'left':
        return 'Reverse()', 'right'

def move_left(orientation):
    if orientation == 'left':
        return 'Forward()', 'left'
    elif orientation == 'up':
        return 'Left() Forward()', 'left'
    elif orientation == 'down':
        return 'Right() Forward()', 'left'
    elif orientation == 'right':
        return 'Reverse()', 'left'

def move_down(orientation):
    if orientation == 'down':
        return 'Forward()', 'down'
    elif orientation == 'right':
        return 'Right() Forward()', 'down'
    elif orientation == 'left':
        return 'Left() Forward()', 'down'
    elif orientation == 'up':
        return 'Reverse()', 'down'

def move_up(orientation):
    if orientation == 'up':
        return 'Forward()', 'up'
    elif orientation == 'left':
        return 'Right() Forward()', 'up'
    elif orientation == 'right':
        return 'Left() Forward()', 'up'
    elif orientation == 'down':
        return 'Reverse()', 'up'

# Implement the A* search algorithm
def a_star_search(grid, src, dest):
    # Check if the source and destination are valid
    if not is_valid(src[0], src[1]) or not is_valid(dest[0], dest[1]):
        print("Source or destination is invalid")
        return

    # Check if the source and destination are unblocked
    if not is_unblocked(grid, src[0], src[1]) or not is_unblocked(grid, dest[0], dest[1]):
        print("Source or the destination is blocked")
        return

    # Check if we are already at the destination
    if is_destination(src[0], src[1], dest):
        print("We are already at the destination")
        return

    # Initialize the closed list (visited cells)
    closed_list = [[False for _ in range(cols)] for _ in range(rows)]
    # Initialize the details of each cell
    cell_details = [[Cell() for _ in range(cols)] for _ in range(rows)]

    # Initialize the start cell details
    i = src[0]
    j = src[1]
    cell_details[i][j].f = 0
    cell_details[i][j].g = 0
    cell_details[i][j].h = 0
    cell_details[i][j].parent_i = i
    cell_details[i][j].parent_j = j

    # Initialize the open list (cells to be visited) with the start cell
    open_list = []
    heapq.heappush(open_list, (0.0, i, j))

    # Initialize the flag for whether destination is found
    found_dest = False

    # Main loop of A* search algorithm
    while len(open_list) > 0:
        # Pop the cell with the smallest f value from the open list
        p = heapq.heappop(open_list)

        # Mark the cell as visited
        i = p[1]
        j = p[2]
        closed_list[i][j] = True

        # For each direction, check the successors
        directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]
        for dir in directions:
            new_i = i + dir[0]
            new_j = j + dir[1]

            # If the successor is valid, unblocked, and not visited
            if is_valid(new_i, new_j) and is_unblocked(grid, new_i, new_j) and not closed_list[new_i][new_j]:
                # If the successor is the destination
                if is_destination(new_i, new_j, dest):
                    # Set the parent of the destination cell
                    cell_details[new_i][new_j].parent_i = i
                    cell_details[new_i][new_j].parent_j = j
                    print("The destination cell is found")
                    # Trace and print the path from source to destination
                    trace_path(cell_details, dest)
                    found_dest = True
                    return
                else:
                    # Calculate the new f, g, and h values
                    g_new = cell_details[i][j].g + 1.0
                    h_new = calculate_h_value(new_i, new_j, dest)
                    f_new = g_new + h_new

                    # If the cell is not in the open list or the new f value is smaller
                    if cell_details[new_i][new_j].f == float('inf') or cell_details[new_i][new_j].f > f_new:
                        # Add the cell to the open list
                        heapq.heappush(open_list, (f_new, new_i, new_j))
                        # Update the cell details
                        cell_details[new_i][new_j].f = f_new
                        cell_details[new_i][new_j].g = g_new
                        cell_details[new_i][new_j].h = h_new
                        cell_details[new_i][new_j].parent_i = i
                        cell_details[new_i][new_j].parent_j = j

    # If the destination cell is not found
    if not found_dest:
        print("Failed to find the destination cell")

# Example grid ('.' is unblocked, 'X' is blocked)
grid = [['.' for _ in range(cols)] for _ in range(rows)]
obstacles = int(input("Enter number of obstacles: "))
print("Enter obstacle positions (row column) separated by space:")

for _ in range(obstacles):
    r, c = map(int, input().split())
    grid[r][c] = 'X'



# Get the source and destination from the user
src = tuple(map(int, input("Enter the source cell (row column): ").split()))
dest = tuple(map(int, input("Enter the destination cell (row column): ").split()))

# Run the A* search algorithm
a_star_search(grid, src, dest)


grid[src[0]][src[1]] = 'A'
grid[dest[0]][dest[1]] = 'B'

# Print the grid
print("The Grid is:")
for row in grid:
    print(' '.join(row))


Enter obstacle positions (row column) separated by space:
The destination cell is found
The Path is 
-> (4, 0) -> (3, 0) -> (3, 1) -> (3, 2) -> (3, 3) -> (3, 4) -> (4, 4) 

Direction of Movement: 
From (4, 0) to (3, 0): Forward()
From (3, 0) to (3, 0): Right()
From (3, 0) to (3, 1): Forward()
From (3, 1) to (3, 2): Forward()
From (3, 2) to (3, 3): Forward()
From (3, 3) to (3, 4): Forward()
From (3, 4) to (3, 4): Right()
From (3, 4) to (4, 4): Forward()
The Grid is:
. . . . .
. . . . .
. . . . .
. . . . .
A X . . B
